### This notebook is to provide an example of the Teradata SQL notebook using the Script Table Operator %runsto magic command. It will download and run a Python script on  Teradata Vantage. 

<b>Note that before running the Script Table Operator commands, you want to make sure that Teradata Vantage has Python libraries installed and that the user has the necessary permissions to install/replace/remove files and to run scripts.</b>

<b>The example script uses the `numpy` Python module. Verify that the `numpy` library is installed by running the `%pyinfo module=NUM` magic command.</b>
<p><b>***NOTE: To execute the SQL or magic command in the cell, press SHIFT+ENTER or the run button in the toolbar.</b>

***

Replace the STODemoUserName, systemName, user, and host values for your system.

In [1]:
%var STODemoUserName=<stodemo username>, systemName=<connection name>, user=<user name>, host=<host name or ip address>

In [ ]:
%lsconnect

### Add a connection definition for Teradata Vantage if it doesn't exist

In [ ]:
%addconnect name=${systemName}, user=${user}, host=${host}

### If not already granted, run the necessary GRANT statements for the STO user.
<b>Note that you must connect to the system as a user that is authorized to grant privileges for the STO user.</b>

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.INSTALL_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.REPLACE_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE PROCEDURE ON SYSUIF.REMOVE_FILE TO ${STODemoUserName};

In [ ]:
GRANT EXECUTE FUNCTION  ON TD_SYSFNLIB.SCRIPT to ${STODemoUserName};

In [ ]:
GRANT EXECUTE ON SYSUIF.DEFAULT_AUTH to ${STODemoUserName};

In [ ]:
GRANT CREATE EXTERNAL PROCEDURE ON stodemo TO stodemo

### Connect 

In [ ]:
%connect ${systemName}

### Show information about the version of Python installed on the active connection

In [ ]:
%pyinfo

### Show information about the version of Python with only the modues starting with "num"

In [ ]:
%pyinfo module=NUM

## Show the help for the runsto magic

In [6]:
%help runsto

#### %runsto
Run a Script Table Operator.

%runsto | %%runsto

* **%%runsto** is a cell magic command and the script that is to be run follows the parameters to the command.
* **%runsto** is a line magic command which runs the script that is already installed on the Vantage system.

The parameters that are used by both the **%runsto** and **%%runsto** variants are:</br></br>
filename=&lt;Name of script file to run (and install for %%runsto).&gt;,</br>
on=&lt;Input to the script, table/view name or query SQL.&gt;,</br>
returns=&lt;Names and data types of values returned by the script.&gt;</br>
[, consumesql=&lt;SQL that consumes the output of the script.&gt;]</br>
[, onoptions=&lt;Options to the ON clause, AS, Hash/Partition/Order.&gt;]</br>
[, delimiter=&lt;Character that delimits fields in input and output.&gt;]</br>
[, charset=&lt;LATIN|UTF-16 (Encoding of data passed to and from script).&gt;]</br>
[, quotechar=&lt;Quotes the input and output data of script.&gt;]</br>
[, auth=&lt;Binds an operating system user to script via authorization objects.&gt;]</br>
[, as=&lt;Alias of output table and, optionally, its columns.&gt;]</br>
[, command=&lt;Linux command to be run with "%s" indicating where the script's name is to be inserted.&gt;]</br></br>
Additional parameters used by **%%runsto**:</br></br>
regname=&lt;Registered name of installed file&gt;</br>
[, replacefile=&lt;true|false&gt;]</br>
[, forcereplace=&lt;true|false&gt;]</br>
script body follows the newline following the parameters.

Default values for optional parameters:</br>
* consumesql – SELECT * FROM
* onoptions – no ON options clause
* delimiter – no DELIMITER clause, database default, tab character, used
* charset – no CHARSET clause, database default LATIN used
* quotechar – no QUOTECHAR clause, database default of not quoting fields
* auth – no AUTH clause
* as – no AS clause
* command – python %s
* replacefile – false
* forcereplace – false

### The cell magic variant of runsto will install the script then run it. The script follows the command line.

### First, make sure the test file is not installed (using the %rmfile magic).

In [ ]:
%rmfile regname=stoTest, prompt=true

### The example script executes a SELECT statement against the `ex2tbl` table, looking for `ObjsID < 400`. You can create a similar table and insert test data or choose a different table and query for your Script Table Operator. Replace "stodemo" with the STO user name you specified above.
***
<pre>
CREATE MULTISET TABLE stodemo.ex2tbl ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      ObsID INTEGER,
      RandSeed FLOAT)
UNIQUE PRIMARY INDEX ( ObsID );

INSERT INTO "stodemo"."ex2tbl"  ("ObsID", "RandSeed")  VALUES("ObsID" [INTEGER ],"RandSeed" [FLOAT ]) ;
***
### Run the following %%runsto magic to install a Python script onto Teradata Vantage.

In [ ]:
%%runsto consumesql = "SELECT COUNT(*) AS nSims, 
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM", \
on=select * from ex2tbl where ObsID < 400,\
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",\
filename=stoTest.py, regname=stoTest, replacefile=false

"""
* The contents of this file are Teradata Public Content 
* and have been released to the Public Domain.
* Please see license.txt file in the package for more information.
* Alexander Kolovos - 4 Sep 2015 - v.1.2
* Copyright 2019-2021 Teradata. All rights reserved.
* Licensed under BSD

R And Python Analytics with the SCRIPT Table Operator
Orange Book supplementary material
Example 2 - Simulation - Python

Simulation Bank renege example taken from:
https://simpy.readthedocs.org/en/latest/examples/bank_renege.html
Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

Requires random, numpy, sys and simpy packages.

Required input:
- table ex2tbl defined in ex2dataTblDef.txt
- ex2tbl data from file ex2data.txt

Input parameters:
INTERVAL_CUSTOMERS : Time interval (minutes) between customer entries
MIN_PATIENCE       : Minimum time (minutes) customers will wait
MAX_PATIENCE       : Maximum time (minutes) customers will wait
TIME_IN_BANK       : Time (minutes) a customer spends being served
MAX_MINUTES        : Process observation time (minutes)

Output:
Number of customers that visited in MAX_MINUTES
Number of customers that reneged in MAX_MINUTES
Average waiting time per customer

"""
import random
import simpy
import numpy as np
import sys

DELIMITER='\t'

# Load data from table, one row at a time
# Expecting table with 2 columns: ObsID, RandomSeed
tbldatalist = []
while 1:
    try:
        line = input()
        if line == '':   # Exit if user provides blank line
            break
        else:
            allnum = line.split(DELIMITER)
            allnum = [x.replace(" ","") for x in allnum]
            tbldatalist.append(allnum)
    except (EOFError):   # Exit if reached EOF or CTRL-D
        break
        
ndl = len(tbldatalist)

# If the present AMP has no data, then exit this script instance.
if ndl == 0:
    sys.exit()

# Assumes data set has ObsID in col 1, RandomSeed in col 2
nDataCols = 2
# Convert list into array
tbldata = np.array(tbldatalist).reshape(ndl,nDataCols)
del tbldatalist

obsID = tbldata[:,0]
randSeed = tbldata[:,1]

# set the variables to default values in case they're not specified on the command line
INTERVAL_CUSTOMERS = 4
MIN_PATIENCE = 5
MAX_PATIENCE = 10
TIME_IN_BANK = 6
MAX_MINUTES = 480

if len(sys.argv) > 1:
    INTERVAL_CUSTOMERS = sys.argv[1]
if len(sys.argv) > 2:
    MIN_PATIENCE = sys.argv[2]
if len(sys.argv) > 3:
    MAX_PATIENCE = sys.argv[3]
if len(sys.argv) > 4:
    TIME_IN_BANK = sys.argv[4]
if len(sys.argv) > 5:
    MAX_MINUTES = sys.argv[5]

RANDOM_SEED = 20           # A default random seed value. Here, seeds are input
INTERVAL_CUSTOMERS = 4.0   # Generate new customers roughly every x minutes
MIN_PATIENCE = 5           # Min. customer patience
MAX_PATIENCE = 10          # Max. customer patience
TIME_IN_BANK = 6.0         # Minutes a customer spends being served
MAX_MINUTES = 480          # Minutes to observe system behavior

def source(env, interval, counter):
    """Source generates customers randomly"""
    global iCust
    iCust = 0
    while 1:
        iCust += 1
        c = customer(env, 'Customer%02d' % iCust, counter, TIME_IN_BANK)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        global totWait
        totWait += wait
        global totReneged

        if req in results:
            # We got to the counter

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)

        else:
            # We reneged
            totReneged += 1

# Setup and start the simulation
for i in range( 0, len(randSeed) ):
    global totWait
    totWait = 0
    global totReneged
    totReneged = 0
    seedNo = randSeed[i]
    random.seed( seedNo )
    env = simpy.Environment()
    
    # Start processes and run
    counter = simpy.Resource(env, capacity=1)
    env.process(source(env, INTERVAL_CUSTOMERS, counter))
    env.run( until=MAX_MINUTES )
    
    print(iCust, DELIMITER, totReneged, DELIMITER, totWait/iCust)


In [ ]:
%showfile stoTest

### Once the file has been installed, it can be run using the %runsto magic without including the script.
#### Run the SCRIPT table operator with only required parameters (script file already installed)
* ON specifies the contents of a table as the input to the script
* RETURNS specifies the output columns
* FILENAME specifies the script file name

All the output from the script is returned (no "consumesql" parameter)
#### In the example below we are referring to an existing table called `ex2tblshort` to provide input to the script. You can create the similar table and load data into it or use one on your own tables.  Replace "stodemo" with the STO user name you specified above.
***
<pre>
CREATE MULTISET TABLE stodemo.ex2tblshort ,NO FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      ObsID INTEGER,
      RandSeed FLOAT)
UNIQUE PRIMARY INDEX ( ObsID );

INSERT INTO "stodemo"."ex2tblshort"  ("ObsID", "RandSeed")  VALUES("ObsID" [INTEGER ],"RandSeed" [FLOAT ]) ;
***

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator as above but with the COMMAND parameter.
#### The "command" parameter is specified because the script is being given command-line arguments. The "%s" indicates where the script file name is to be inserted into the conmand.

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py,
command= "/opt/teradata/languages/Python/bin/python3 %s 4 5 10 6 480"

### Run the SCRIPT table operator as before except count and average the script output columns
#### The "consumesql" parameter specifies a query that will read the script's output data

In [ ]:
%runsto 
consumesql = "SELECT COUNT(*) AS nSims,
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM",
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator with the results of a query as its input
#### The "on" parameter specifies a query

In [ ]:
%runsto 
consumesql = "SELECT COUNT(*) AS nSims,
       AVG(CAST (oc1 AS INT)) AS AvgCustomers, 
       AVG(CAST (oc2 AS INT)) AS AvgReneged,
       AVG(CAST (oc3 AS FLOAT)) AS AvgWaitTime FROM",
on=select * from ex2tbl where ObsID < 400,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

### Run the SCRIPT table operator and insert its output into another table.
#### The "consumesql" parameter specifies an INSERT/SELECT that inserts the results of the script into a table.

In [ ]:
drop table ${STODemoUserName}.STAGE;

In [ ]:
CREATE TABLE ${STODemoUserName}.STAGE
     (
      c1 varchar(10),
      c2 varchar(10),
      c3 varchar(50));

In [ ]:
%runsto
consumesql = "INSERT INTO ${STODemoUserName}.STAGE(c1, c2, c3) SELECT * FROM",
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=stoTest.py

In [18]:
SELECT * FROM ${STODemoUserName}.STAGE;

,c1,c2,c3
1,133,67,5.140471076255329
2,144,77,5.505535341901332
3,120,59,5.33958339300195
4,160,71,5.277195186186821
5,158,79,5.127128900489865
6,114,36,4.593148218276117
7,93,38,4.423803503608627
8,131,45,4.620375491969863
9,127,62,4.962570791819159
10,130,53,4.8748724327807675


### The following two cells illustrate what is being run when the %runsto magic is run

In [ ]:
SET SESSION SEARCHUIFDBPATH = "${STODemoUserName}"

In [ ]:
SELECT * FROM 
SCRIPT(
      ON ex2tblshort
      SCRIPT_COMMAND('/opt/teradata/languages/Python/bin/python3 ./${STODemoUserName}/stoTest.py')
      RETURNS ('oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)')
)

### Run a script that contains a syntax error. The lines in the script log from the server that are the result of running the script are shown.
#### The example below assumes there is an `ex2pERR.py` file on the system. You can use the above example `stoTest.py` and force an error in the script to demonstrate the error handling.

In [ ]:
%runsto 
on=ex2tblshort,
returns = "oc1 VARCHAR(10), oc2 VARCHAR(10), oc3 VARCHAR(50)",
filename=ex2pERR.py

In [ ]:
%showfile ex2pERR

### Script files (and data files) can be installed without running them using the %installfile magic

In [24]:
%help installfile

#### %installfile
Install a file onto the Vantage system.

%installfile regname=&lt;Registered name of installed file&gt;,</br>filename=&lt;Name of file in Teradata&gt;,</br>localfile=&lt;Name of the file to be installed&gt;</br>[, replace=&lt;true|false&gt;]</br>[, forcereplace=&lt;true|false&gt;]</br>[, text=&lt;true|false&gt;]

* The **regname** is used when replacing, removing and showing the file.
* The **filename** is used by scripts that reference the file.
* The **localfile** is the name of the file on your file system that is to be installed.
* The **replace** parameter indicates whether the file is being initially installed (replace=false) or being replaced (replace=true).
* The **forcereplace** parameter indicates whether the file is to be replaced even if it is currently being used.
* The **text** parameter indicates if the file is a text file (true, the default value) or a binary file (false).

#### Run the %rmfile to remove the file in case it is already installed.

In [ ]:
%rmfile regname=stoTest2

#### Install the file
The localfile parameter value can include the full path name of the file or it can be relative to the current directory.
If the file is a binary file, you will need to set the text parameter value to false.
#### The example below assumes there is a Python file called `ex2p.py` on your local machine.

In [ ]:
%installfile regname=stoTest2, filename=stoTest2.py, localfile=ex2p.py, text=true

#### If the file gets modified and needs to be replaced, include the "replace=true" parameter.
The text parameter can be omitted for text files.

In [ ]:
%installfile regname=stoTest2, filename=stoTest2.py, localfile=ex2p.py, replace=true

#### Remove the file.
The prompt=false parameter says that confirmation isn't required'

In [ ]:
%rmfile regname=stoTest2, prompt=false

Copyright 2019-2022 Teradata. All rights reserved.